In [1]:
import pandas as pd
import numpy as np
from contractions import contractions
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
from generatedata import generate_data 

In [2]:
data = generate_data('bigData/yelp_review.csv', 10)

Reading CSV....(Might take around 1:30mins to 2mins- 3.5GB file)
Reading Done.
Data Generated


In [2]:
# def readAllDataAndMerge():
#     data = pd.read_csv('dataset/avg_cleaned_25k_yelp.csv')
#     posReviews = pd.read_csv('dataset/pos_cleaned_25k_yelp.csv')
#     negReviews = pd.read_csv('dataset/neg_cleaned_25k_yelp.csv')
    
#     data = data.append(posReviews, ignore_index = True) 
#     data = data.append(negReviews, ignore_index = True) 
#     return data

In [3]:
# data = readAllDataAndMerge()

In [4]:
data.head()

,stars,date,text,useful
1,4,2014-12-28,"Cool hotel, came here for an anniversary a lit...",2
2,5,2017-03-18,"What can I say . Excellent service , you get e...",1
3,4,2014-05-22,"I got a few macarons, a tart and a cappuccino....",0
4,5,2015-02-15,"Oh yah, this is my childhood crush from the 70...",0
5,4,2015-03-18,"Great place, for it was here that I finally fo...",0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 526168 entries, 1 to 526168
Data columns (total 4 columns):
stars     526168 non-null int64
date      526168 non-null object
text      526168 non-null object
useful    526168 non-null int64
dtypes: int64(2), object(2)
memory usage: 20.1+ MB


In [6]:
def assignPolarityToReviews(stars):
    if stars > 3:
        return 1
    if stars == 3:
        return 0
    return -1

In [7]:
starsColumn = data['stars']
polarity = starsColumn.map(assignPolarityToReviews)
data['polarity'] = polarity #1: Good | 0:Neutral | 1:Bad

In [8]:
data = data.sample(frac=1).reset_index(drop=True)
data.head(10)

,stars,date,text,useful,polarity
0,5,2015-06-07,Tai and her stylist Carol did my hair and make...,3,1
1,4,2017-04-19,I HIGHLY recommend Sara if you are looking for...,5,1
2,4,2015-10-23,I didn't know going to Costco could become a h...,3,1
3,4,2014-05-31,A bit pricey but consistently delicious!!! Ser...,0,1
4,5,2016-06-21,Ashley gave me a tour of the apartment and she...,0,1
5,5,2016-07-23,Hauled my tukas all the way out to Monroeville...,1,1
6,5,2012-12-12,"Finally, civilization arrives in Champaign-Urb...",1,1
7,5,2015-11-27,A rough november but Today has started of as a...,0,1
8,5,2017-02-18,So went into buck and badger last Saturday for...,0,1
9,1,2017-11-27,This place is horrible at everything. The bart...,0,-1


In [9]:
#Stopwords selection
#https://www.cs.cmu.edu/~ark/EMNLP-2015/proceedings/WASSA/pdf/WASSA14.pdf

stopWords = set(stopwords.words('english'))
print(stopWords)

{'yourselves', "wouldn't", 'a', 'he', 'now', 'at', 'so', 'his', 'both', 'we', 'were', 'had', 'about', 'my', "you'll", "she's", "weren't", 'into', 'these', 'have', 'in', 'mightn', 'o', 'won', 'was', 'before', 'that', 'on', 'not', 'or', 'can', 'the', 'when', 'below', 'being', 'over', 'other', 'themselves', 'me', 'how', 'ain', 'hers', 'but', 'been', 'didn', "shan't", 'with', 'once', 'just', "won't", 'more', 'off', "aren't", 'down', 'for', 'then', 'ma', 'only', 'myself', 'y', 'hasn', 'yours', 'ourselves', 'does', 'further', 'aren', 'haven', 'is', 'nor', 've', 'from', 'while', 'our', 'by', 'did', 'm', 'through', 'having', 'after', 'couldn', 'out', 'shouldn', 'theirs', 'because', "don't", 'don', 'where', 'why', 'most', 'any', 'should', "needn't", 'own', 'if', 'her', 'there', "shouldn't", 'of', 'each', "couldn't", 'it', 'between', 'and', 'those', 'll', 'needn', 'weren', 'isn', 'wouldn', 'which', "didn't", "you'd", "wasn't", 'they', "doesn't", 't', 'whom', 'against', 'she', 'you', "hadn't", 'w

In [10]:
def filterStopWords(words):
    filteredList = set()
    for word in words:
        if "n't" not in word:
            filteredList.add(word)
    return filteredList



stopWords = filterStopWords(stopWords)
print(stopWords)

{'yourselves', 'a', 'he', 'now', 'at', 'so', 'his', 'both', 'we', 'were', 'had', 'about', 'my', "you'll", "she's", 'into', 'these', 'have', 'in', 'mightn', 'o', 'won', 'was', 'before', 'that', 'on', 'not', 'or', 'can', 'the', 'when', 'below', 'over', 'being', 'other', 'themselves', 'me', 'how', 'ain', 'hers', 'but', 'been', 'didn', 'with', 'once', 'just', 'more', 'off', 'down', 'for', 'then', 'ma', 'only', 'myself', 'y', 'yours', 'ourselves', 'does', 'further', 'aren', 'haven', 'is', 'nor', 've', 'from', 'while', 'our', 'by', 'did', 'm', 'through', 'having', 'after', 'couldn', 'out', 'shouldn', 'theirs', 'because', 'don', 'where', 'why', 'most', 'any', 'should', 'own', 'if', 'her', 'there', 'of', 'each', 'it', 'between', 'and', 'doing', 'those', 'll', 'needn', 'weren', 'isn', 'wouldn', 'which', "you'd", 'they', 't', 'whom', 'against', 'she', 'you', 'wasn', 'all', 'him', 'himself', 'i', 'what', "you've", 'doesn', "should've", 're', 'some', 'them', 'shan', 'be', 'its', 'same', 'd', "it's

In [11]:

wordsRequiredOfStopWords = ['not', 'nor', 'neither' , 'no']

stopWords = stopWords.difference(wordsRequiredOfStopWords)

In [12]:
#Improving performance in string manipulation
def expandContractions(text):
    words = text.split(' ')
    expandedTextList = []
    for word in words:
        if(word in contractions):
            expandedTextList.append(contractions[word].split('/')[0]) #Always considering 1st form of verb for list
        else:
            expandedTextList.append(word)
        expandedTextList.append(' ')
    return ''.join(expandedTextList)

def getNonStopWordsLowerCase(text):
    textTokens = word_tokenize(text)
    retText = []
    for w in textTokens:
        w = w.lower()
        if(w not in stopWords and len(w) > 1) :
            retText.append(w + ' ')
            
    return ''.join(retText)

def textCleaning(text):
    
    #Remove Hyperlinks
    # Regex: r'\S+', # sequence of non-whitespace
    text = re.sub(r'http\S+', '', text)
    
    #Remove all tags
    text = BeautifulSoup(text, 'html.parser').get_text()
    
    #Expand texts
    text = expandContractions(text)
    
    #Remove all words containing numbers
    #https://stackoverflow.com/questions/18082130/python-regex-to-remove-all-words-which-contains-number
    text = ' '.join(s for s in text.split() if not any(c.isdigit() for c in s))
    
    #Remove special characters
    text = re.sub('[^A-Za-z]+', ' ', text)
    
    #Get non stop words in lower case
    text = getNonStopWordsLowerCase(text).strip()
    
    return text


In [13]:

preprocessed_texts = []

for text in tqdm(data['text'].values):
    doneText = textCleaning(text)
    #if(len(doneText) < 2):
    #   continue
    preprocessed_texts.append(doneText)


 95%|████████████████████████████████████████████████████████████████████    | 497513/526168 [05:25<00:18, 1529.45it/s]C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:302: UserWarning: "b'II own a unit at the Maya and am totally shocked by the negative vicious review of it by someone..... .I love living here. It is VERY convenient to stores, highways, cultural events, etc. Everything in my unit works well - I have no complaints\\/..'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
100%|████████████████████████████████████████████████████████████████████████| 526168/526168 [05:44<00:00, 1528.04it/s]


In [14]:
len(preprocessed_texts)

526168

In [15]:
#Sample prints

def printPreprocessedTexts(idx):

    print("Original Text:\t",data['text'][idx] , end='\n\n')
    print("Preprocessed Text:\t",preprocessed_texts[idx], end='\n\n')

    print("Polarity of the text: ", data['polarity'][idx])

In [16]:
printPreprocessedTexts(0)

Original Text:	 Tai and her stylist Carol did my hair and make up for my wedding.   Both of them are absolutely awesome!!! They're extremely professional, prompt and are top notch in their respective trades.  We were transformed in an hour and had fun along the way.  Tai has a portable make up chair,  air brush and every color imaginable. 

I would recommend Tai &  her team to anyone who wants professional hair & make up done in Vegas, they're the best!

Preprocessed Text:	 tai stylist carol hair make wedding absolutely awesome extremely professional prompt top notch respective trades transformed hour fun along way tai portable make chair air brush every color imaginable would recommend tai team anyone wants professional hair make done vegas best

Polarity of the text:  1


In [17]:
printPreprocessedTexts(1)

Original Text:	 I HIGHLY recommend Sara if you are looking for a great, talented hair colorist! She saved my terribly damaged hair from another hair salon's bad coloring job.
She put beautiful highlights and lowlights on my hair. I showed her a picture of what I wanted and she did EXACTLY what it looked like on my hair. No one ever could've done that because I have super thick, coarse hair that colors or bleach usually don't go in very easily. On top of that, I have a ton of hair which took Sara 3 hours to put high and low lights! She is very thorough. I don't think anyone was this immaculate putting color on my hair. She makes sure everyone leaves the salon happy but she would never do anything to damage or ruin your hair. I saw Sara's beautiful platinum blond work on someone and it was so shiny and healthy looking. 
Not only she is very talented, her personality is amazing!! I always get a little nervous spending hours in hair salons and  having conversation with hair stylists (Well,

In [18]:
printPreprocessedTexts(2)

Original Text:	 I didn't know going to Costco could become a hobby, but now I do. I come here with no intention of buying anything but it is Costco, and you're bound to walk out of this place with something. There is something for everyone!

I bought a floral arrangement here for $30, where at a florists it would be at least $90+ (if you've purchased an arrangement from a floral shop, you know what I'm talking about). Also, the samples spread throughout this place are amazing, omg I can't stop eating when I'm here. It is a good sales tactic, you try it and love it and want more... ugh so good.

The pizza is delicious and cheap. Out of curiosity, I looked up the calorie count on the pizza. ONE SLICE = 700 CALORIES. Luckily, my friend and I split one piece down the middle (350 calories but still, wow). BEWARE.

Preprocessed Text:	 not know going costco could become hobby come no intention buying anything costco bound walk place something something everyone bought floral arrangement flori

In [19]:
#Writing to CSV to avoid recomputation

#colList = ['Review','Polarity','Useful']
to_save = pd.DataFrame({'Review': preprocessed_texts, 'Polarity': data['polarity'], 'Useful':data['useful'] })
to_save.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 526168 entries, 0 to 526167
Data columns (total 3 columns):
Review      526168 non-null object
Polarity    526168 non-null int64
Useful      526168 non-null int64
dtypes: int64(2), object(1)
memory usage: 12.0+ MB


In [20]:
to_save.to_csv('dataset/preprocessed.csv', index=False)